# Functional Programming in Python
## David Mertz
### dmertz@continuum.io
### 2016-04-22

<hr/>
This tutorial, and Python in general, run more smoothly under Python 3.x.

Whether you're running on Python 2 or Python 3, please install [Python-Future](http://python-future.org/futurize.html):
```bash
conda install future
```

In [ ]:
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)
from future import standard_library
standard_library.install_aliases()
from future.builtins import (
         bytes, dict, int, list, object, range, str,
         ascii, chr, hex, input, next, oct, open,
         pow, round, super, filter, map, zip)

# Table of Contents
* [Decorator Functions](#Decorator-Functions)
	* [Decorator classes](#Decorator-classes)
	* [Memoization](#Memoization)

# Decorator Functions

Decorators are a very powerful functionality of Python.
A very simple decorator (the identity decorator)

In [ ]:
def decorate(func):
    def new_func(arg):
        return func(arg)
    return new_func

In [ ]:
@decorate
def a(b):
    return b+1

<img src="img/decorator.png" width="300px" height="300px" />

Understanding decorators is straight forward.  A decorator is a function that accepts a function as input and returns another function.  This function that is returned from the decorator replaces the original, decorated function.  In the example above, ```new_func``` becomes the new implementation of ```a```.  The decorator doesn't do anything special right now.  However, decorators are extremely handy for wrapping a function.

Let's imagine that we want to allow ```a``` to operate on sequences of numbers.

In [ ]:
def map_scalar(func):
    def map_to_seq(*args):
        return map(func, args)
    return map_to_seq

In [ ]:
@map_scalar
def add_one(x):
    return x + 1

print(list(add_one(3)))
print(list(add_one(10, 20, 30, 40)))

We can also pass arguments to decorators.  In order to accept these parameters, we have to wrap our decorator in another function that will accept these.  Let's write a decorator that will write the output of a function to a file.

In [ ]:
def function_log(fn):
    def wrapped(func):
        def new_func(*args, **kwargs):
            out = func(*args, **kwargs)
            with open(fn, 'w') as fo:
                fo.write(out)
            return out
        return new_func
    return wrapped

In [ ]:
logfile="tmp/myfunc.log"
@function_log(logfile)
def myfunc(a, b, c=None):
    outstr = "{} is the value of a\n{} is the value of b\n\n{}".format(a, b, c)
    return outstr

<img src="img/decorator_args.png" width=800px height=800px/>

In [ ]:
print(myfunc(3, 6, "None"))

In [ ]:
# Lets read the log
with open(logfile, 'r') as fi:
    print(''.join(fi.readlines()))

In [ ]:
def add_two(func):
    def _(arg):
        return func(arg) + 2
    return _

In [ ]:
# We can apply more than one decorator by stacking them
# add_one is replaced by add_two and then replaced by map_scalar
@map_scalar
@add_two
def add_one(x):
    return x + 1

print(list(add_one(*([1]*3))))

## Decorator classes

We can even use a class to define a decorator.  We need to define the ```__call__()``` dunder method.  It works exactly the same as the function decorator.

In [ ]:
class logfile(object):
    def __init__(self, fn):
        # These are the arguments accepted 
        self.fn = fn
        
    def __call__(self, func):
        def _(*args, **kwargs):
            out = func(*args, **kwargs)
            with open(self.fn, 'w') as fo:
                fo.write(out)
            return out
        return _

@logfile('tmp/myfunc2.log')
def g():
    return 'hello from function g'
g()


## Memoization

Suppose we have a computationally intensive method, ```f()```, that calculates some result (in this case, a number).
We have to call this function many times, but do not wait forever to recalculate our result.

In [ ]:
import time

def f(a, b):
    # <expensive number crunching here>
    out = a + b
    time.sleep(1.5)
    return out

In [ ]:
%timeit f(3, 5)

Wouldn't it be nice to be able to cache the results of our previous calls so when we call ```f()``` again with the same arguments we could simply return the cached result instead of recalculating the answer?  Of course, it would be very nice!

This sort of caching is called _memoization_.  Lets define a class decorator that will memoize any function that we decorate

In [ ]:
class Memoizer(object):
    def __init__(self, func):
        self.cache = {}
        self.func = func
        
    def __call__(self, *args, **kwargs):
        # We use sorted tuples because they are much smaller in memory than frozensets
        # Even though frozensets are slightly faster to construct.
        # We sort so that we can compare the keyword args.
        # We use strings because our args or kwargs may not be hashable.
        # Repr should return a unique string for its object
        key = (repr(args), repr(tuple(sorted(kwargs.items()))))
        if key in self.cache:
            return self.cache[key]
        self.cache[key] = self.func(*args, **kwargs)
        return self.cache[key]

In [ ]:
@Memoizer
def f(a, b):
    # <expensive number crunching here>
    out = a + b
    time.sleep(1.5)
    return out

In [ ]:
f([3], [5, 5])

In [ ]:
f.cache

In [ ]:
import continuum_style; continuum_style.style()